In [ ]:
!pip install openpyxl

In [ ]:
import yaml
import random
import numpy as np
import time

from reader import Reader
from const import Const
from utils import get_F_example
from clustering import Clusters
from i_merge import IMerger

from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

# Если есть желание создать свои данные, то вызвать этот код

In [ ]:
X, y = make_blobs(n_samples=500, centers=7, cluster_std=0.6, random_state=0)

ids = [i+1 for i in range(len(X))]
df = pd.DataFrame()
df['id'] = ids
df['X1'] = X[:,0]
df['X2'] = X[:,1]

In [ ]:
ids = []
X1 = []
X2 = []
with open('Two.txt', 'r') as f:
    for k, line in enumerate(f):
        if k==0:
            continue
        ids.append(int(line.split()[0]))
        X1.append(float(line.split()[1]))
        X2.append(float(line.split()[2]))
        if k>=205:
            break
df = pd.DataFrame()
df['id'] = ids
df['X1'] = X1
df['X2'] = X2
    

In [ ]:
reader = Reader()

In [ ]:
df = reader.read('input.xlsx')

In [ ]:
df.head()

In [ ]:
plt.scatter(df['X1'], df['X2'])
plt.show()

# Подбор констант

In [ ]:
const = Const('./example.yaml')
# const = Const()

In [ ]:
const.norm(df) # Нормировка

In [ ]:
const.get_norms() # Получение норм

In [ ]:
a = time.time()
const.calculate_a(df, type_of_optimization=3) # Вычисление а и пересчет констант
b = time.time()
print(b-a)

In [ ]:
const.add_Fcolumn(df)

In [ ]:
df.head()

In [ ]:
const.config # Вывод параметров

In [ ]:
# const.config['conturs']['min_diff'] = 6.21
const.config['isolated_cluster']['divider'] = 1

In [ ]:
# const.save_consts('example') #Сохранение констант в файл

Researching dataset

In [ ]:
fig = px.scatter_3d(df, x='X1', y='X2', z='F')
fig.show()

# Кластеризация

In [ ]:
df.head()

In [ ]:
cluster = Clusters(const.config) 

In [ ]:
df = cluster.get_isolated_clusters(df) # Вычисление кластеров
df.head()

In [ ]:
set(df['cluster_id'])

In [ ]:
for cluster in set(df['cluster_id']):
    plt.scatter(df[df['cluster_id']==cluster]['X1'], df[df['cluster_id']==cluster]['X2'])


In [ ]:
fig = px.scatter_3d(df, x='X1', y='X2', z='F',
              color='cluster_id')
fig.show()

In [ ]:
df.head()

# Merge I

In [ ]:
Merger = IMerger(const.config)
Merger.mergeClusters(df)

In [ ]:
plt.scatter(df['X1'],df['X2'], c=df['cluster_id'])

In [ ]:
fig = px.scatter_3d(df, x='X1', y='X2', z='F',
              color='cluster_id')
fig.show()

# Subcluster

In [ ]:
first = df[df['cluster_id']==1].sort_values('F')

In [ ]:
print(first)

In [ ]:
fig = px.scatter_3d(first, x='X1', y='X2', z='F',
              color='id')
fig.show()

In [ ]:
const.config

In [ ]:
def get_profile(F, p1, p2, divider = 1, min_diff = 3.48, max_div_num = 8, min_len =  0.23, a = 0.23, min_dif=3.48):
    #Функция рассчета профиля F - матрица формата ['id','x1',...,'xn','F'], p1, p2 - точки формата ['id','x1',...,'xn','F']
    if np.linalg.norm(np.array(p1[1:-1]) - np.array(p2[1:-1]))/divider <= min_diff:
        return 'close', 0
    else:
        div_num = 0
        segment_len = np.linalg.norm( np.array(p1[1:-1]) - np.array(p2[1:-1]))
        while (div_num < max_div_num-1) and (segment_len > min_len):
            div_num+= 1
            num_of_segments = (divider+1)*div_num + divider
            segment_len = np.linalg.norm(np.array(p1[1:-1]) - np.array(p2[1:-1]))/num_of_segments

        if div_num == 0:
            return 'close', 0

        points = []
        for i in range(num_of_segments):
            x = [-1.]
            for j in range(1, len(p1)-1):
                x.append((p1[j] + p2[j]*(i+1)/(num_of_segments-i))/(1+(i+1)/(num_of_segments-i)))
            points.append(np.array(x))

        Fs = [[point[1],  point[2], get_F_example([f[:-1] for f in F], const.config['consts']['a'], target=point)] \
                                                                                                for point in points]
        Fs = sorted(Fs, key = lambda S: S[-1], reverse = False)
#         print(Fs)

        Fmin = np.min([F[-1] for F in Fs])
        Fstar = p1[-1] if  p1[-1] < p2[-1] else p2[-1]

        if Fstar - Fmin >= min_dif:
            return 'different', Fstar - Fmin
        else:
            return 'common', Fstar - Fmin

In [ ]:
X = first.values
F_matrix = X[:,:-1]
F_matrix.shape

In [ ]:
step = 10
layers_F = []
current = step
while current<X[:,-2].max():
    layers_F.append(current)
    current += step
layers_F

In [ ]:
def calculate_distance(p, matrix):
    """
    p - [id, X1,...,Xn,F, cluster_id]
    matrix - [[id, X1,...,Xn,F, cluster_id],...,[id, X1,...,Xn,F, cluster_id]]
    ========
    return - index of nearest neighbor
    """
    start = True
    for row_index, row in enumerate(matrix):
        if p[0]==row[0]:
            continue
        distance = np.linalg.norm(np.array(p[1:-2]) - np.array(row[1:-2]))
        if start:
            min_distanse = distance
            min_row_index = row_index
            start = False
        if min_distanse > distance:
#             print(min_distanse)
            min_distanse = distance
            min_row_index = row_index
    return min_row_index

In [ ]:
def check_levels(p, connected, layers):
    """
    p - [id, X1,...,Xn,F, cluster_id]
    connected - list of points [id, X1,...,Xn,F, cluster_id]
    layers - list
    ========
    return - bool value, point is closed or not
    """
    if len(connected) == 0:
        return True
    curret_layer = None
    for i in range(len(layers)):
        if p[-2]<layers[i]:
            curret_layer = layers[i-1]
            break
    if curret_layer is None:
        curret_layer = layers[-1]
    for p_c in connected:
        if p_c[-2]>curret_layer:
            return False
    return True
    

In [ ]:
def calculate_distance_for_2_points(A,B):
    """
    A - [id, X1,...,Xn,F, cluster_id]
    B - [id, X1,...,Xn,F, cluster_id]
    =========
    return - distance between 2 points
    """
    return np.linalg.norm(np.array(A[1:-2]) - np.array(B[1:-2]))

In [ ]:
def merge(A,B, F_matrix):
    """
    A - list of points [id, X1,...,Xn,F, cluster_id]
    B - list of points [id, X1,...,Xn,F, cluster_id]
    F_matrix - [[id, X1,...,Xn,F],...,[id, X1,...,Xn,F]]
    =======
    return - bool value, status for merging clusters
    """
    for p_1 in A:
        for p_2 in B:
            result, _ = get_profile(F_matrix, p_1[:-1], p_2[:-1])
#             print(result)
            if result == 'different':
                return False
    return True

In [ ]:
subcluster_result = {int(x[0]):None for x in X}
connected_points = {int(x[0]):[] for x in X}
closed_points = []
subcluster = 0
start = True
for k,p in enumerate(X):
    print('###'*30)
    print('CURRENT POINT', p)
    if k == X.shape[0]-1:
        index = calculate_distance(p,X)
        subcluster_result[int(p[0])] = subcluster_result[int(X[index][0])]
        nearest_p = X[index]
        result, value = get_profile(F_matrix, p[:-1], nearest_p[:-1]) #calculate profile
    else:
        index = calculate_distance(p,X[k:]) # find nearest neighbor
        result, value = get_profile(F_matrix, p[:-1], X[k+index,:-1]) #calculate profile
        nearest_p = X[k+index]
    print('NEAREST POINT', nearest_p)
#     plt.scatter(X[:,1],X[:,2], c='b')
#     plt.scatter(nearest_p[1],nearest_p[2], c='r')
#     plt.scatter(p[1],p[2], c='g')
#     plt.show()
    print(result)
    if result != 'common' and result != 'close': # Если точки разные
        if subcluster_result[int(p[0])] == None:
            if not start:
                subcluster += 1
            subcluster_result[int(p[0])] = subcluster
            
    else: # Если точки близки или профиль прошел проверку
        #Обе точки не имеют подкластера-создаем новый
        
        if subcluster_result[int(p[0])] == None and subcluster_result[int(nearest_p[0])] == None:
            print("TOGETHER NONE")
            if not start:
                subcluster += 1
            subcluster_result[int(p[0])] = subcluster
            subcluster_result[int(nearest_p[0])] = subcluster
            
        #Одна из точек не имеет подкластера или они не совпадают
        elif subcluster_result[int(p[0])] != subcluster_result[int(nearest_p[0])]:
            print("DIFFERENT SUB")
#             if k == 96 or k == 97:
#             print(subcluster_result)
#             print(subcluster_result[int(p[0])])
#             print(subcluster_result[int(nearest_p[0])])
            A = [p]
            B = [nearest_p]
            #Находим все точки принадлежащие данным подкластерам
            for point in closed_points:
                if subcluster_result[int(point[0])] == subcluster_result[int(p[0])] and subcluster_result[int(p[0])]!= None:
                    A.append(point)
                if subcluster_result[int(point[0])] == subcluster_result[int(nearest_p[0])] and \
                    subcluster_result[int(nearest_p[0])]!= None:
                    B.append(point)
#             if k == 96 or k == 97:
#             plt.figure(figsize=(15,15))
#             plt.scatter(X[:,1],X[:,2], c='w')
#             plt.scatter(np.stack(A)[:,1],np.stack(A)[:,2], c='r')
#             plt.scatter(np.stack(B)[:,1],np.stack(B)[:,2], c='g')
#             plt.show()
#             print('A')
#             for a in A:
#                 print(a)
#             print('B')
#             for b in B:
#                 print(b)
            # Проверяем их на слияние
            if not merge(A,B, F_matrix):#Если разные, то проверяем куда отнести ближайшую точку
#                 if k == 206 or k == 97:
                print(k, 'NOT MERGE')
                if subcluster_result[int(p[0])] == None:
                    if not start:
                        subcluster += 1
                    subcluster_result[int(p[0])] = subcluster
                status = True
                start_dist = calculate_distance_for_2_points(A[0],B[0])
                for i in range(1,len(B)):
                    if calculate_distance_for_2_points(B[0],B[i])<start_dist:
                        status = False
                        break
                if status:
                    subcluster_result[int(nearest_p[0])] = subcluster_result[int(p[0])]
            else:#Если подкластеры сливаются - то всем точкам анализируемого подкластера сливаются
#                 if k == 96 or k == 97:
                print(k, 'MERGE')
                if subcluster_result[int(nearest_p[0])] is None:
                    subcluster_result[int(nearest_p[0])] = subcluster_result[int(p[0])]
                else:
                    if subcluster_result[int(p[0])] is None:
                        subcluster_result[int(p[0])] = subcluster_result[int(nearest_p[0])]
                    else:
                        deleted_cluster = subcluster_result[int(p[0])]
                        for p_name in subcluster_result:
                            if subcluster_result[p_name] == deleted_cluster:
                                subcluster_result[p_name] = subcluster_result[int(nearest_p[0])]
            
        connected_points[int(nearest_p[0])].append(p)# Добавляем в список точек текущую
        
#         connected_points[int(nearest_p[0])] += connected_points[int(p[0])]
    start = False
    closed_points.append(p)
#     print('subcluster_result')
#     print(subcluster_result)
#     print(subcluster_result[int(p[0])])
#     print(subcluster_result[int(nearest_p[0])])
#     print('closed_points')
#     print(closed_points)
#     if check_levels(p, connected_points[int(p[0])], layers_F):
#         closed_points.append(p)


#     color = []
#     for c in list(subcluster_result.values()):
#         if c == None:
#             color.append(-1)
#         else:
#             color.append(c)
#     plt.scatter(X[:,1],X[:,2], c=color)
#     plt.show()
#     print('*^#'*30)
#     for x in X:
        
#         plt.annotate(str(subcluster_result[int(x[0])]), xy = (x[1], x[2]))
    
#     YYY = np.stack(closed_points)
#     plt.scatter(YYY[:,1],YYY[:,2], marker='^')
    plt.show()
    
print('*'*30)
print(len(connected_points))
print('*'*30)
print(subcluster_result)
print(len(subcluster_result))
print('*'*30)
print(len(closed_points))

In [ ]:
set(subcluster_result.values())

In [ ]:
fig, ax = plt.subplots(figsize=(13, 13))
ax.scatter(X[:,1],X[:,2], c=list(subcluster_result.values()))
for x in X:
    ax.annotate(str(int(x[0])), xy = (x[1]+0.02, x[2]))
    
YYY = np.stack(closed_points)
for y in YYY:
    circle2 = plt.Circle((y[1], y[2]), 0.08, color='b', fill=False)
    ax.add_patch(circle2)
ax.plot()   #Causes an autoscale update.
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(X[:,1],X[:,2], c=list(subcluster_result.values()))
for x in X:
    plt.annotate(str(int(x[0])), xy = (x[1]+0.02, x[2]))
    

In [ ]:
fig, ax = plt.subplots(figsize=(13, 13))
ax.scatter(X[:,1],X[:,2], c=list(subcluster_result.values()))
for x in X:
    ax.annotate(str(subcluster_result[int(x[0])]), xy = (x[1]+0.02, x[2]))
    
YYY = np.stack(closed_points)
for y in YYY:
    circle2 = plt.Circle((y[1], y[2]), 0.08, color='b', fill=False)
    ax.add_patch(circle2)
ax.plot()   #Causes an autoscale update.
plt.show()

# Merging check

In [ ]:
def merge(A,B, F_matrix):
    """
    A - list of points [id, X1,...,Xn,F, cluster_id]
    B - list of points [id, X1,...,Xn,F, cluster_id]
    F_matrix - [[id, X1,...,Xn,F],...,[id, X1,...,Xn,F]]
    =======
    return - bool value, status for merging clusters
    """
    for p_1 in A:
        for p_2 in B:
            result, _ = get_profile(F_matrix, p_1[:-1], p_2[:-1])
#             print(result)
            if result == 'different':
                print(p_1)
                print(p_2)
                return False
    return True

In [ ]:
cluster_1 = 39
cluster_2 = 27
A = []
B = []
for p in X:
    if subcluster_result[int(p[0])] == cluster_1:
        A.append(p)
    if subcluster_result[int(p[0])] == cluster_2:
        B.append(p)
    

In [ ]:
merge(A,B,F_matrix)

# Profile Check

In [ ]:
for x in X:
    if x[0] == 144:
        p1 = x
    if x[0] == 185:
        p2 = x

In [ ]:
p1

In [ ]:
p2

In [ ]:
p1[1:-2]

In [ ]:
np.linalg.norm(np.array(p1[1:-2]) - np.array(p2[1:-2]))

In [ ]:
np.linalg.norm(np.array(p1[1:-2]) - np.array(p2[1:-2]))

In [ ]:
np.linalg.norm(np.array(p1[1:-2]) - np.array(p2[1:-2]))

In [ ]:
get_profile(F_matrix, p1[:-1], p2[:-1])

In [ ]:
plt.scatter(X[:,1],X[:,2], c='b')
plt.scatter(p1[1],p1[2], c='r')
plt.scatter(p2[1],p2[2], c='g')

# Grapth

In [ ]:
import plotly.express as px

In [ ]:
plot_df = first.copy()

In [ ]:
subcluster = []
for ind in plot_df['id'].values:
    subcluster.append(subcluster_result[ind])
plot_df['sub'] = subcluster
closed = []
closed_find = np.stack(closed_points)
for ind in plot_df['id'].values:
    if ind in closed_find[:,0]:
        closed.append(10)
    else:
        closed.append(1)
plot_df['closed'] = closed  

In [ ]:
plot_df.head()

In [ ]:
fig = px.scatter(plot_df, x="X1", y="X2", color="sub", hover_data=['id', 'F'], )

In [ ]:
fig.show()